In [1]:
import numpy as np
import pandas as pd
import os
import gym
import random
from gym import spaces
import torch
from Environment import DiagnosisEnv
from Agent import train_DQN

In [2]:
df_symptoms = pd.read_csv("Data/Symptoms.csv")
df_exams = pd.read_csv("Data/Examinations.csv")
df_emergency_diagnoses = pd.read_csv("Data/Emergency_Diagnosis.csv")

In [3]:
env = DiagnosisEnv(df_symptoms, df_exams, df_emergency_diagnoses)
agent = train_DQN(env)

Epsiode 0, Total reward: 0.30, Epsilon: 1.000
Epsiode 10, Total reward: -1.40, Epsilon: 0.951


C:\Users\jerem\Project_Hospital\Agent.py:28: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  return (torch.tensor(state, dtype=torch.float32),


Epsiode 20, Total reward: -1.00, Epsilon: 0.905
Epsiode 30, Total reward: -1.00, Epsilon: 0.860
Epsiode 40, Total reward: 0.20, Epsilon: 0.818
Epsiode 50, Total reward: 0.90, Epsilon: 0.778
Epsiode 60, Total reward: 0.50, Epsilon: 0.740
Epsiode 70, Total reward: -1.90, Epsilon: 0.704
Epsiode 80, Total reward: 0.40, Epsilon: 0.670
Epsiode 90, Total reward: 0.50, Epsilon: 0.637
Epsiode 100, Total reward: 0.40, Epsilon: 0.606
Epsiode 110, Total reward: 0.50, Epsilon: 0.576
Epsiode 120, Total reward: 0.70, Epsilon: 0.548
Epsiode 130, Total reward: 0.40, Epsilon: 0.521
Epsiode 140, Total reward: 0.60, Epsilon: 0.496
Epsiode 150, Total reward: 0.60, Epsilon: 0.471
Epsiode 160, Total reward: 0.50, Epsilon: 0.448
Epsiode 170, Total reward: -1.30, Epsilon: 0.427
Epsiode 180, Total reward: 0.70, Epsilon: 0.406
Epsiode 190, Total reward: 0.50, Epsilon: 0.386
Epsiode 200, Total reward: -1.20, Epsilon: 0.367
Epsiode 210, Total reward: 0.80, Epsilon: 0.349
Epsiode 220, Total reward: -1.10, Epsilon: 

In [4]:
def make_diagnosis(agent, env, diagnosis_labels):
    with torch.no_grad():
        state = torch.tensor(env.observed_symptoms, dtype=torch.float32).unsqueeze(0)
        similarities = []
        for diagnosis, symptoms_dict in env.diagnoses:
            true_symptom_vec = np.zeros(len(env.symptoms))
            for sym, prob in symptoms_dict.items():
                index = env.symptom_to_index[sym]
                true_symptom_vec[index] = prob
            similarities.append((diagnosis, np.dot(state[0].numpy(), true_symptom_vec)))
        guess = max(similarities, key=lambda x: x[1])[0]
        return env.make_diagnosis(guess)

In [5]:
def test_agent(env, agent, num_patients=5):
    correct_diagnoses = 0

    for patient_num in range(num_patients):
        state = env.reset()

        print(f"\n🩺 PATIENT {patient_num + 1}")
        print(f"🧑‍⚕️ TRUE DIAGNOSIS: {env.get_true_diagnosis()}")
        print(f"✅ TRUE SYMPTOMS (Generated for this patient): {env.get_true_symptoms()}\n")

        exams_taken = []
        symptoms_history = []

        done = False
        step_count = 0

        while not done:
            # ----- Use your trained agent to select the best exam -----
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
            with torch.no_grad():
                q_values = agent(state_tensor).squeeze()  # Output shape: (num_examinations,)
            action = q_values.argmax().item()
            exam_taken = env.examinations[action]

            # ----- Environment step -----
            state, reward, done, info = env.step(action)
            observed_symptoms = env.get_observed_symptoms()

            exams_taken.append(exam_taken)
            symptoms_history.append(observed_symptoms)

            print(f"🔎 Step {step_count + 1}:")
            print(f"   ➡️ Examination taken: {exam_taken}")
            print(f"   🧬 Symptoms uncovered so far: {observed_symptoms}")

            # If forced to diagnose due to repeated exam
            if info.get("forced", False):
                print("\n⚠️ Exam was repeated! Forced to diagnose.")
                print(f"🔮 FINAL DIAGNOSIS GUESS: {info['guess']}")
                print(f"🎯 TRUE DIAGNOSIS: {info['diagnosis']}")
                print(f"🏆 Reward: {reward}")

                if info["guess"] == info["diagnosis"]:
                    correct_diagnoses += 1
                break

            step_count += 1

        if not info.get("forced", False):
            print("\n🚩 Max steps reached or no diagnosis made.")
            print(f"Examinations taken: {exams_taken}")
            print(f"Observed symptoms: {observed_symptoms}")

    accuracy = correct_diagnoses / num_patients
    print(f"\n📊 Accuracy over {num_patients} patients: {accuracy:.2f}")

In [6]:
test_agent(env, agent, num_patients=10)


🩺 PATIENT 1
🧑‍⚕️ TRUE DIAGNOSIS: bronchitis
✅ TRUE SYMPTOMS (Generated for this patient): ['chills', 'fever', 'headache', 'wheezing', 'bronchial infection']

🔎 Step 1:
   ➡️ Examination taken: interview
   🧬 Symptoms uncovered so far: ['chills', 'fever', 'headache', 'wheezing']
🔎 Step 2:
   ➡️ Examination taken: x ray back
   🧬 Symptoms uncovered so far: ['chills', 'fever', 'headache', 'wheezing']
🔎 Step 3:
   ➡️ Examination taken: x ray back
   🧬 Symptoms uncovered so far: ['chills', 'fever', 'headache', 'wheezing']

⚠️ Exam was repeated! Forced to diagnose.
🔮 FINAL DIAGNOSIS GUESS: bronchitis
🎯 TRUE DIAGNOSIS: bronchitis
🏆 Reward: 1

🩺 PATIENT 2
🧑‍⚕️ TRUE DIAGNOSIS: bronchitis
✅ TRUE SYMPTOMS (Generated for this patient): ['breathing problems', 'chest pain', 'chills', 'dry cough', 'fatigue', 'wheezing', 'bronchial infection']

🔎 Step 1:
   ➡️ Examination taken: interview
   🧬 Symptoms uncovered so far: ['breathing problems', 'chest pain', 'chills', 'dry cough', 'fatigue', 'wheezing'